In [3]:
### 라이브러리 및 데이터 불러오기
# 필요한 라이브러리를 불러온다.
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import pickle
import torchvision

# 데이터 전처리 방식을 지정한다.
transform = transforms.Compose([
        transforms.Resize((100,100)),
        transforms.ToTensor(), # 데이터를 PyTorch의 Tensor 형식으로 바꾼다.
        transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)) # 픽셀값 0 ~ 1 -> -1 ~ 1
])
train_data = torchvision.datasets.ImageFolder(root='/home/capstone_ai1/kong/403_High_Small/',
                                        transform=transform,
                                        )


# 데이터를 한번에 batch_size만큼만 가져오는 dataloader를 만든다.
dataloader = DataLoader(train_data, batch_size=100, shuffle=True)

In [4]:
import os
import imageio

if torch.cuda.is_available():
    use_gpu = True
leave_log = True
if leave_log:
    result_dir = 'DCGAN_generated_images'
    if not os.path.isdir(result_dir):
        os.mkdir(result_dir)

In [5]:
c = train_data[0][0].shape[0]
h = train_data[0][0].shape[1]
w = train_data[0][0].shape[2]

print("c:", c,"h:",h,"w:",w)
print(len(train_data))

c: 3 h: 100 w: 100
11178


In [6]:
### DCGAN의 생성자
class Generator(nn.Module):
    
    # 네트워크 구조
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            ## layer1
            nn.ConvTranspose2d(in_channels=100, out_channels=1024, 
                               kernel_size=4, stride=1, padding=0, 
                               bias=False),
            nn.BatchNorm2d(num_features=1024),
            nn.ReLU(inplace=True),
            ## layer2
            nn.ConvTranspose2d(in_channels=1024, out_channels=512, 
                               kernel_size=4, stride=2, padding=1, 
                               bias=False),
            nn.BatchNorm2d(num_features=512),
            nn.ReLU(True),
            ## layer3
            nn.ConvTranspose2d(in_channels=512, out_channels=256, 
                               kernel_size=8, stride=2, padding=1, 
                               bias=False),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(True),
            
            ## layer4
            nn.ConvTranspose2d(in_channels=256, out_channels=128, 
                   kernel_size=8, stride=2, padding=1, 
                   bias=False),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(True),
            ## layer5
            nn.ConvTranspose2d(in_channels=128, out_channels=3, 
                   kernel_size=16, stride=2, padding=1, 
                   bias=False),          
            nn.Tanh())
        
    # (batch_size x 100) 크기의 랜덤 벡터를 받아 
    # 이미지를 (batch_size x 1 x 28 x 28) 크기로 출력한다.
    def forward(self, inputs):
        inputs = inputs.view(-1, 100, 1, 1)
        return self.main(inputs)

In [7]:
from torchsummary import summary
model = Generator()
model = model.cuda()
summary(model, (60,100))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1           [-1, 1024, 4, 4]       1,638,400
       BatchNorm2d-2           [-1, 1024, 4, 4]           2,048
              ReLU-3           [-1, 1024, 4, 4]               0
   ConvTranspose2d-4            [-1, 512, 8, 8]       8,388,608
       BatchNorm2d-5            [-1, 512, 8, 8]           1,024
              ReLU-6            [-1, 512, 8, 8]               0
   ConvTranspose2d-7          [-1, 256, 20, 20]       8,388,608
       BatchNorm2d-8          [-1, 256, 20, 20]             512
              ReLU-9          [-1, 256, 20, 20]               0
  ConvTranspose2d-10          [-1, 128, 44, 44]       2,097,152
      BatchNorm2d-11          [-1, 128, 44, 44]             256
             ReLU-12          [-1, 128, 44, 44]               0
  ConvTranspose2d-13          [-1, 3, 100, 100]          98,304
             Tanh-14          [-1, 3, 1

In [8]:
### DCGAN의 구분자
class Discriminator(nn.Module):
    
    # 네트워크 구조
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            ##layer1
            nn.Conv2d(in_channels=3, out_channels=128, 
                      kernel_size=16, stride=2, padding=1, 
                      bias=False),
            nn.BatchNorm2d(num_features=128), #input에 안써야(Generator는 output) stable 해짐
            nn.LeakyReLU(0.2, inplace=True),
            ##layer2
            nn.Conv2d(in_channels=128, out_channels=256, 
                      kernel_size=8, stride=2, padding=1, 
                      bias=False),
            nn.BatchNorm2d(num_features=256),
            nn.LeakyReLU(0.2, inplace=True),
            ##layer3
            nn.Conv2d(in_channels=256, out_channels=512, 
                      kernel_size=8, stride=2, padding=1, 
                      bias=False),
            nn.BatchNorm2d(num_features=512),
            nn.LeakyReLU(0.2, inplace=True),
            ##layer4
            nn.Conv2d(in_channels=512, out_channels=1024, 
                      kernel_size=4, stride=2, padding=1, 
                      bias=False),
            nn.BatchNorm2d(num_features=1024),
            nn.LeakyReLU(0.2, inplace=True),
            ##layer5
            nn.Conv2d(in_channels=1024, out_channels=1, 
                      kernel_size=4, stride=1, padding=0, 
                      bias=False),
            nn.Sigmoid())
        
    # (batch_size x 1 x 28 x 28) 크기의 이미지를 받아
    # 이미지가 진짜일 확률을 0~1 사이로 출력한다.
    def forward(self, inputs):
        o = self.main(inputs)
        return o.view(-1, 1)

In [9]:
from torchsummary import summary
model = Discriminator()
model = model.cuda()
summary(model, (3, 100, 100))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 44, 44]          98,304
         LeakyReLU-2          [-1, 128, 44, 44]               0
            Conv2d-3          [-1, 256, 20, 20]       2,097,152
       BatchNorm2d-4          [-1, 256, 20, 20]             512
         LeakyReLU-5          [-1, 256, 20, 20]               0
            Conv2d-6            [-1, 512, 8, 8]       8,388,608
       BatchNorm2d-7            [-1, 512, 8, 8]           1,024
         LeakyReLU-8            [-1, 512, 8, 8]               0
            Conv2d-9           [-1, 1024, 4, 4]       8,388,608
      BatchNorm2d-10           [-1, 1024, 4, 4]           2,048
        LeakyReLU-11           [-1, 1024, 4, 4]               0
           Conv2d-12              [-1, 1, 1, 1]          16,384
          Sigmoid-13              [-1, 1, 1, 1]               0
Total params: 18,992,640
Trainable para

In [10]:
### 생성자와 구분자 객체 만들기
G = Generator()
D = Discriminator()

if use_gpu:
    G.cuda()
    D.cuda()

In [11]:
### 손실 함수와 최적화 기법 지정하기
# Binary Cross Entropy loss
criterion = nn.BCELoss()

# 생성자의 매개 변수를 최적화하는 Adam optimizer
G_optimizer = Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
# 구분자의 매개 변수를 최적화하는 Adam optimizer
D_optimizer = Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [12]:
# 학습 결과 시각화하기
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

def square_plot(data, path):
    """Take an array of shape (n, height, width) or (n, height, width , 3)
       and visualize each (height, width) thing in a grid of size approx. sqrt(n) by sqrt(n)"""

#     if type(data) == list:
# 	    data = np.concatenate(data)

    # normalize data for display
    data = (data - data.min()) / (data.max() - data.min())

    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))

    padding = (((0, n ** 2 - data.shape[0]) ,
                (0, 1), (0, 1))  # add some space between filters
               + ((0, 0),) * (data.ndim - 3))  # don't pad the last dimension (if there is one)
    data = np.pad(data , padding, mode='constant' , constant_values=1)  # pad with ones (white)

    # tilethe filters into an image
    data = data.reshape((n , n) + data.shape[1:]).transpose((0 , 2 , 1 , 3) + tuple(range(4 , data.ndim + 1)))

    data = data.reshape((n * data.shape[1] , n * data.shape[3]) + data.shape[4:])

    plt.imsave(path, data)

In [13]:
if leave_log:
    train_hist = {}
    train_hist['D_losses'] = []
    train_hist['G_losses'] = []
    generated_images = []
    
z_fixed = torch.randn(5 * 5, 100)
if use_gpu:
    z_fixed = z_fixed.cuda()

In [14]:
# if use_gpu:
#     real_data, target_real, target_fake = real_data.cuda(), target_real.cuda(), target_fake.cuda()

# 진짜 이미지를 구분자에 넣는다.
for real_data, _ in dataloader:
    real_data = real_data.cuda()
    batch_size = real_data.size(0)
    break
print(real_data.shape)
D_result_from_real = D(real_data)
print(D_result_from_real.shape)
z = torch.randn((batch_size, 100))
if use_gpu:
    z = z.cuda()

# 생성자로 가짜 이미지를 생성한다.
fake_data = G(z)
print(fake_data.shape)

torch.Size([100, 3, 100, 100])
torch.Size([100, 1])
torch.Size([100, 3, 100, 100])


In [15]:
### 모델 학습을 위한 반복문
# 데이터셋을 100번 돌며 학습한다.
for epoch in range(100):
    
    if leave_log:
        D_losses = []
        G_losses = []
    
    for real_data, _ in dataloader:
        print("ing")
        batch_size = real_data.size(0) # 60
        # 데이터를 pytorch의 변수로 변환한다.
#         real_data = Variable(real_data)

        ### 구분자 학습시키기

        # 이미지가 진짜일 때 정답 값은 1이고 가짜일 때는 0이다.
        # 정답지에 해당하는 변수를 만든다.
        target_real = torch.ones(batch_size, 1)
        target_fake = torch.zeros(batch_size, 1)
         
        if use_gpu:
            real_data, target_real, target_fake = real_data.cuda(), target_real.cuda(), target_fake.cuda()
            
        # 진짜 이미지를 구분자에 넣는다.
        D_result_from_real = D(real_data)
        # 구분자의 출력값이 정답지인 1에서 멀수록 loss가 높아진다.
        D_loss_real = criterion(D_result_from_real, target_real)

        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = torch.randn((batch_size, 100))
        
        if use_gpu:
            z = z.cuda()
            
        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)
#         print(fake_data.shape)
        # 생성자가 만든 가짜 이미지를 구분자에 넣는다.
        D_result_from_fake = D(fake_data)
        # 구분자의 출력값이 정답지인 0에서 멀수록 loss가 높아진다.
        D_loss_fake = criterion(D_result_from_fake, target_fake)
        
        # 구분자의 loss는 두 문제에서 계산된 loss의 합이다.
        D_loss = D_loss_real + D_loss_fake
        
        # 구분자의 매개 변수의 미분값을 0으로 초기화한다.
        D.zero_grad()
        # 역전파를 통해 매개 변수의 loss에 대한 미분값을 계산한다.
        D_loss.backward()
        # 최적화 기법을 이용해 구분자의 매개 변수를 업데이트한다.
        D_optimizer.step()
        
        if leave_log:
            D_losses.append(D_loss.data.item())

        # train generator G

        ### 생성자 학습시키기
        
        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = torch.randn((batch_size, 100))
        
        if use_gpu:
            z = z.cuda()
        
        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)
#         print("fake_data:",fake_data.shape)
        # 생성자가 만든 가짜 이미지를 구분자에 넣는다.
        D_result_from_fake = D(fake_data)
        # 생성자의 입장에서 구분자의 출력값이 1에서 멀수록 loss가 높아진다.
        G_loss = criterion(D_result_from_fake, target_real)
        
        # 생성자의 매개 변수의 미분값을 0으로 초기화한다.
        G.zero_grad()
        # 역전파를 통해 매개 변수의 loss에 대한 미분값을 계산한다.
        G_loss.backward()
        # 최적화 기법을 이용해 생성자의 매개 변수를 업데이트한다.
        G_optimizer.step()
        
        if leave_log:
            G_losses.append(G_loss.data.item())
    if leave_log:
        true_positive_rate = (D_result_from_real > 0.5).float().mean().data.item()
        true_negative_rate = (D_result_from_fake < 0.5).float().mean().data.item()
        base_message = ("Epoch: {epoch:<3d} D Loss: {d_loss:<8.6} G Loss: {g_loss:<8.6} "
                        "True Positive Rate: {tpr:<5.1%} True Negative Rate: {tnr:<5.1%}"
                       )
        message = base_message.format(
                    epoch=epoch,
                    d_loss=sum(D_losses)/len(D_losses),
                    g_loss=sum(G_losses)/len(G_losses),
                    tpr=true_positive_rate,
                    tnr=true_negative_rate
        )
        print(message)

    if leave_log:
        fake_data_fixed = G(z_fixed)
        fake_data_fixed = fake_data_fixed.transpose(1 , 2)
        fake_data_fixed = fake_data_fixed.transpose(2 , 3)
        fake_data_fixed = fake_data_fixed.cpu().data.numpy()
        image_path = result_dir + '/epoch{}.png'.format(epoch)
        
        
        square_plot(fake_data_fixed, image_path)
        generated_images.append(image_path)
    
    if leave_log:
        train_hist['D_losses'].append(torch.mean(torch.FloatTensor(D_losses)))
        train_hist['G_losses'].append(torch.mean(torch.FloatTensor(G_losses)))

torch.save(G.state_dict(), "dcgan_generator.pkl")
torch.save(D.state_dict(), "dcgan_discriminator.pkl")
with open('dcgan_train_history.pkl', 'wb') as f:
    pickle.dump(train_hist, f)

generated_image_array = [imageio.imread(generated_image) for generated_image in generated_images]
imageio.mimsave(result_dir + '/DCGAN_generation.gif', generated_image_array, fps=5)

ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
Epoch: 0   D Loss: 0.503469 G Loss: 7.38233  True Positive Rate: 100.0% True Negative Rate: 100.0%
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
Epoch

ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
Epoch: 15  D Loss: 0.771209 G Loss: 3.47836  True Positive Rate: 93.6% True Negative Rate: 91.0%
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
Epoch: 

ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
Epoch: 30  D Loss: 0.590169 G Loss: 4.45612  True Positive Rate: 98.7% True Negative Rate: 98.7%
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
Epoch: 

ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
Epoch: 45  D Loss: 0.6422   G Loss: 4.87217  True Positive Rate: 87.2% True Negative Rate: 100.0%
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
Epoch:

ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
Epoch: 60  D Loss: 0.583206 G Loss: 4.0528   True Positive Rate: 56.4% True Negative Rate: 85.9%
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
Epoch: 61  

ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
Epoch: 76  D Loss: 0.572783 G Loss: 4.36495  True Positive Rate: 92.3% True Negative Rate: 100.0%
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
Epoch: 77  D Loss: 0.521196 G Loss: 4.18847  True Positive Rate: 100.0% True Negative Rate: 100.0%
ing
ing
ing
ing
ing
ing
ing
ing

ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
Epoch: 91  D Loss: 0.544839 G Loss: 3.93361  True Positive Rate: 97.4% True Negative Rate: 100.0%
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
ing
Epoch: 92  D Loss: 0.316622 G Loss: 4.57031  True Positive Rate: 97.4% True Negative Rate: 100.0%
ing
ing
ing
ing
ing
ing
ing
ing
